In [1]:
import pandas as pd
from pynytimes import NYTAPI
NYT_API_KEY='e1RpBU3JBdcJ0dWyz6mB61EiXWG1DsM8'

In [2]:
nyt = NYTAPI(NYT_API_KEY, parse_dates=True)

In [3]:
data = pd.read_csv('data/ai-perception.csv')

In [4]:
article_title = data['Title'][0]
print(article_title)
article_paragraph = data[data['Title'] == article_title]['Paragraph'].values[0]

LONG ISLAND JOURNAL


In [5]:
query = nyt.article_search(query=article_title, results=1)

In [6]:
# print query keys
print(query[0].keys())

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])


In [7]:
query[0]

{'abstract': "Long Island Journal notes controversy in Oakdale over roadside memorial to two teenagers killed in car crash; photo; Patchogue's new regulation of outdoor pay phones also noted (S)",
 'web_url': 'https://www.nytimes.com/1996/12/29/nyregion/long-island-journal-882046.html',
 'snippet': "Long Island Journal notes controversy in Oakdale over roadside memorial to two teenagers killed in car crash; photo; Patchogue's new regulation of outdoor pay phones also noted (S)",
 'lead_paragraph': 'Memorial Mired in Dispute',
 'print_section': 'LI',
 'print_page': '13',
 'source': 'The New York Times',
 'multimedia': [],
 'headline': {'main': 'LONG ISLAND JOURNAL',
  'kicker': 'LONG ISLAND JOURNAL',
  'content_kicker': None,
  'print_headline': 'LONG ISLAND JOURNAL',
  'name': None,
  'seo': None,
  'sub': None},
 'keywords': [{'name': 'glocations',
   'value': 'Long Island (NY)',
   'rank': 1,
   'major': 'N'},
  {'name': 'glocations', 'value': 'Oakdale (NY)', 'rank': 2, 'major': 'N'}

In [8]:
# query[0]['headline']['main']
url = query[0]['web_url']

metadata = nyt.article_metadata(url)
metadata

KeyboardInterrupt: 

In [ ]:
for article in query:
    if article.title == article_title:
        article_contents = article.abstract

AttributeError: 'dict' object has no attribute 'title'